In [ ]:
import openmm
import openmmtools
import simtk.unit as unit
import pathlib
import endstate_correction
from endstate_correction.system import create_charmm_system
from openmm.app import CharmmParameterSet, CharmmPsfFile, CharmmCrdFile, PDBFile, Simulation
from openmm import Platform
from rdkit import Chem
from typing import List

system_name = 'ZINC00107550'              

def setup():
    path = pathlib.Path(endstate_correction.__file__).resolve().parent
    hipen_testsystem = f"{path}/data/hipen_data"
    path_to_psf = f"{hipen_testsystem}/{system_name}/{system_name}.psf"
    path_to_crd = f"{hipen_testsystem}/{system_name}/{system_name}.crd"
    psf = CharmmPsfFile(path_to_psf)
    coord = CharmmCrdFile(path_to_crd)
    params = CharmmParameterSet(
        f"{hipen_testsystem}/top_all36_cgenff.rtf",
        f"{hipen_testsystem}/par_all36_cgenff.prm",
        f"{hipen_testsystem}/{system_name}/{system_name}.str",
    )
    # create openmm simulation system
    return psf.createSystem(params, nonbondedMethod=openmm.app.NoCutoff, constraints=None)



In [ ]:
system = setup()

In [ ]:
from openmm import XmlSerializer
with open('ZINC00107550.xml', 'w') as output:
    output.write(XmlSerializer.serialize(system))



In [ ]:
path = pathlib.Path(endstate_correction.__file__).resolve().parent
hipen_testsystem = f"{path}/data/hipen_data"
path_to_psf = f"{hipen_testsystem}/{system_name}/{system_name}.psf"
path_to_crd = f"{hipen_testsystem}/{system_name}/{system_name}.crd"
psf = CharmmPsfFile(path_to_psf)
coord = CharmmCrdFile(path_to_crd)

with open("temp.pdb", "w") as outfile:
    PDBFile.writeFile(psf.topology, coord.positions, outfile)
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
# Create the RDKit molecule
rdkit_mol = Chem.MolFromPDBFile("temp.pdb", removeHs=False)
#rdkit_mol = Chem.AddHs(rdkit_mol)
AllChem.Compute2DCoords(rdkit_mol)
# Visualize the molecule
for i, atom in enumerate(rdkit_mol.GetAtoms()):
    # For each atom, set the property "molAtomMapNumber" to a custom number, let's say, the index of the atom in the molecule
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

img = Draw.MolToImage(rdkit_mol)
img


In [ ]:
def _set_torsion(simulation: Simulation, dihedral_indices:dict, torsion_idx:int, restraint, phase_offset:float):
    #print(restraint.getNumTorsions())
    #print(restraint.getTorsionParameters(torsion_idx))
    dihedral_index_i, dihedral_index_j, dihedral_index_k, dihedral_index_l = dihedral_indices[torsion_idx]
    restraint.setTorsionParameters(torsion_idx, 
                                dihedral_index_i,
                                dihedral_index_j,
                                dihedral_index_k,
                                dihedral_index_l,
                                1,
                                phase_offset, 
                                k=50*unit.kilojoules_per_mole
                                )
    #print(restraint.getTorsionParameters(torsion_idx))
    restraint.updateParametersInContext(simulation.context)

def _get_energy(simulation: Simulation):
    state = simulation.context.getState(getEnergy = True)
    return state.getPotentialEnergy() / unit.kilojoules_per_mole

def _get_position(simulation: Simulation):
    state = simulation.context.getState(getPositions=True)
    return state.getPositions()
    
def torsion_scan_1D(dihedral_indices:dict, ncycles:int, restraint, simulation:Simulation)-> List[float]:
    
    energies = []
    torsion_idx = 0
    sim_e = setup()
    integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 1*unit.femtoseconds)
    simulation_e = mm.app.Simulation(psf.topology, sim_e, integrator)

    for theta in np.linspace(-np.pi, np.pi, ncycles):    
        #simulation.context.setPositions(coord.positions)
        _set_torsion(simulation, dihedral_indices, torsion_idx, restraint, theta* unit.radian)
        #simulation.step(100)
        simulation.minimizeEnergy()
        pos = _get_position(simulation)

        simulation_e.context.setPositions(pos)
        simulation_e.context.setVelocitiesToTemperature(300*unit.kelvin)
        energy = _get_energy(simulation_e)
        energies.append(energy)
    return energies

def torsion_scan_2D(dihedral_indices:dict, ncycles:int, restraint, simulation:Simulation)-> List[float]:
    sim_e = setup()
    integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 1*unit.femtoseconds)
    simulation_e = mm.app.Simulation(psf.topology, sim_e, integrator)

    torsion_idx_i = 0
    torsion_idx_j = 1
    energy_2d = []
    for cycle_i, theta_i in enumerate(np.linspace(-np.pi, np.pi, ncycles)):  
        #simulation.context.setPositions(coord.positions)
        _set_torsion(simulation, dihedral_indices, torsion_idx_i, restraint, theta_i * unit.radian)
        energies = []
        for cycle_j, theta_j in enumerate(np.linspace(-np.pi, np.pi, ncycles)):  
            #simulation.context.setPositions(coord.positions) 
            _set_torsion(simulation, dihedral_indices, torsion_idx_j, restraint, theta_j * unit.radian)
            #simulation.step(100)
            simulation.minimizeEnergy(tolerance=1*unit.kilojoules_per_mole/unit.nanometer)
            pos = _get_position(simulation)
            PDBFile.writeFile(simulation.topology, pos, file=open(f'mconfs/conf_{cycle_i:02d}_{cycle_j:02d}_{theta_i:4f}_{theta_j:4f}.pdb', 'w'))
            # Set the positions and velocities of the particles
            simulation_e.context.setPositions(pos)
            #simulation_e.context.setVelocitiesToTemperature(300*unit.kelvin)

            energies.append(_get_energy(simulation_e))


        energy_2d.append(energies)
        
        
    return energy_2d


In [ ]:
with open("temp.pdb", "w") as outfile:
    PDBFile.writeFile(psf.topology, coord.positions, outfile)
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
# Create the RDKit molecule
rdkit_mol = Chem.MolFromPDBFile("temp.pdb", removeHs=False)
#rdkit_mol = Chem.AddHs(rdkit_mol)
AllChem.Compute2DCoords(rdkit_mol)
# Visualize the molecule
for i, atom in enumerate(rdkit_mol.GetAtoms()):
    # For each atom, set the property "molAtomMapNumber" to a custom number, let's say, the index of the atom in the molecule
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

img = Draw.MolToImage(rdkit_mol)
img


In [ ]:
# 2D torsion plot
import simtk.openmm as mm
import simtk.unit as unit
import openmmtools
from matplotlib.pylab import plt
import os
import numpy as np

#os.removedirs('mconfs')
#os.mkdir('mconfs')
#dihedral_indices = [(11,10,7,9),(7,6,4,3)]
dihedral_indices = [(1,2,7,8), (7,8,9,17)]
mm_system = setup()
# Create a TorsionForce to handle the torsion angle restraints
restraint = mm.PeriodicTorsionForce()
# Add the TorsionForce to the system
mm_system.addForce(restraint)

# Add the torsion angle restraints to the system
for dihedral_idx, (idx_i, idx_j, idx_k, idx_l) in enumerate(dihedral_indices):
    restraint.addTorsion(idx_i, idx_j, idx_k, idx_l, 1, 0*unit.radians,100*unit.kilojoule_per_mole)

# Set up the integrator
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 1*unit.femtoseconds)

# Set up the simulation object
simulation = mm.app.Simulation(psf.topology, mm_system, integrator)

# Set the positions and velocities of the particles
simulation.context.setPositions(coord.positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

# Perform a torsion angle scan
ncycles = 20
energies = torsion_scan_2D(dihedral_indices, ncycles, restraint, simulation)
import seaborn as sns
from matplotlib.pyplot import figure
figure(figsize=(15.0, 15.0), dpi=600)

axs = sns.heatmap(energies, annot=True, cmap="Spectral")#fmt=".1f",linewidth=.5)
axs.set_yticklabels([f"{v:.1f}" for v in np.linspace(-np.pi, np.pi, ncycles)])
axs.set_ylabel(r'$\chi_{1}$', fontsize=18)
axs.set_xlabel(r'$\chi_{2}$', fontsize=18)
axs.set_xticklabels([f"{v:.1f}" for v in np.linspace(-np.pi, np.pi, ncycles)])
#axs.set_xlabel('')
plt.show()

In [ ]:
import simtk.openmm as mm
import simtk.unit as unit
import openmmtools
from matplotlib.pylab import plt
import os
import numpy as np
import seaborn as sns
f = open('../torsion_scan_geometrics_openMM/scan-final.xyz')
i,j = 0,0
energy = np.zeros((60,60))
for l in f.readlines():
    if 'Energy' in l:
        energy[i][j] = float(l.split(' ')[-1])
        j += 1
        
        if j == 60:
            j = 0
            i += 1
            
axs = sns.heatmap(energy, annot=False, cmap="Spectral")#fmt=".1f",linewidth=.5)
axs.set_yticklabels([f"{v:.1f}" for v in np.linspace(-180, 180, 20)])
axs.set_ylabel(r'$\chi_{1}$', fontsize=18)
axs.set_xlabel(r'$\chi_{2}$', fontsize=18)
axs.set_xticklabels([f"{v:.1f}" for v in np.linspace(-180, 180, 20)])
#axs.set_xlabel('')
plt.show()